In [1]:
from web3 import Web3

In [2]:
w3 = Web3()

In [6]:
url = 'https://eth-goerli.g.alchemy.com/v2/fss_yq1JH8COJapGjbQCuaCD77JrjQRp'

In [7]:
web3 = Web3(Web3.HTTPProvider(url))

In [8]:
print(web3.isConnected())

True


In [9]:
print(web3.eth.blockNumber)

7533653


In [10]:
web3.eth.gasPrice

200000000

In [13]:
account_payer = '0xA5b372E60a60A70f2c6ACB87710eba30Ecc4D479'
account_receiver = '0xA059250F4b97bbB3C081f6D9e1fC7249c6Ea2A0c'

In [14]:
account_payer = Web3.toChecksumAddress(account_payer)
account_receiver = Web3.toChecksumAddress(account_receiver)

In [15]:
gas_price = web3.eth.gasPrice

In [17]:
gas = web3.eth.estimateGas({'from': account_payer,
                            'to': account_receiver})

In [24]:
nonce = web3.eth.getTransactionCount(account_payer)

In [26]:
value = 0.01

In [27]:
tx = {'nonce': nonce,
      'to': account_receiver,
      'value': web3.toWei(value, 'ether'),
      'gas': gas,
      'gasPrice': gas_price,
      'chainId': 5}

In [28]:
signed_tx = web3.eth.account.signTransaction(tx, '217d5c81a38240e09ee05bc4ab3efce2eb91d24973162f9833ed18aa7b4460b9')

In [29]:
tx_hash = web3.eth.sendRawTransaction(signed_tx.rawTransaction)

In [30]:
print(web3.toHex(tx_hash))
print(signed_tx)
print(signed_tx.rawTransaction)

0x5d3598453def719bc7fdeb0c75d0cd95dbda0b9ba3ce358016e036454a2ba3b7
SignedTransaction(rawTransaction=HexBytes('0xf86a80840bebc1fe82520894a059250f4b97bbb3c081f6d9e1fc7249c6ea2a0c872386f26fc10000802ea08abbfa10fb6f47c5c5993a344da10b73d078360b5c99d41b2dcff3bbc8313abfa04055d7c9d15cf7e3eedf30457532bcd374ebee2a6e21076ec7e32f6ba07989f4'), hash=HexBytes('0x5d3598453def719bc7fdeb0c75d0cd95dbda0b9ba3ce358016e036454a2ba3b7'), r=62751299400029327074816709941091852717848774607773819814621011024221104585407, s=29099693626297565500403209639187510424098202437303851845503626674786279197172, v=46)
b'\xf8j\x80\x84\x0b\xeb\xc1\xfe\x82R\x08\x94\xa0Y%\x0fK\x97\xbb\xb3\xc0\x81\xf6\xd9\xe1\xfcrI\xc6\xea*\x0c\x87#\x86\xf2o\xc1\x00\x00\x80.\xa0\x8a\xbb\xfa\x10\xfboG\xc5\xc5\x99:4M\xa1\x0bs\xd0x6\x0b\\\x99\xd4\x1b-\xcf\xf3\xbb\xc81:\xbf\xa0@U\xd7\xc9\xd1\\\xf7\xe3\xee\xdf0Eu2\xbc\xd3t\xeb\xee*n!\x07n\xc7\xe3/k\xa0y\x89\xf4'


In [32]:
k = input(prompt='The account address of the payer.')
print(1 + 2)
d = input(prompt='dd')


The account address of the payer.df
3
dddf


In [33]:
"""
Filename: utils.py
Author: Ziyu Ye

This file can be seen as a package for KEM + SKE en/decryption.
"""

import os
import oqs
import time
import json
import base64
import binascii
import argparse

from pathlib import Path
from pprint import pprint
from collections import OrderedDict
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC


def bytes_to_str(text_bytes):
    """
    Transform a bytes in string format.
    Input: text (bytes)
    Return: string format of the bytes
    """
    return binascii.hexlify(text_bytes).decode('utf-8')


def str_to_bytes(text_str):
    """
    Transform a string in bytes format.
    Input: string (bytes)
    Return: bytes format of the bytes
    """
    return binascii.unhexlify(text_str.encode('utf-8'))


def key_transform(key: bytes=b'\xfe',
                  salt: bytes=b'5\xa3'):
    """
    This function transforms an KEM key to the format
    of 32 url-safe base64-encoded bytes.

    Input:
        key (bytes in binary format): the key from KEM
        salt (bytes): a string generated by os.urandom(16)
    Return:
        key_ (bytes in base64 format): the key used for SKE

    Source: cryptography.io/en/latest/
            fernet/#using-passwords-with-fernet
    """

    password = binascii.hexlify(key)
    kdf = PBKDF2HMAC(algorithm=hashes.SHA256(),
                     length=32,
                     salt=salt,
                     iterations=480000)
    key_ = base64.urlsafe_b64encode(kdf.derive(password))

    return key_


def hybrid_encryption(message: str='8964',
                      kem_algo: str='Kyber512'):
    """
    This function encrypts the message using KEM (Key Encryption
    Mechanism) and SKE (Symmetric Key Encryption).

    Inputs:
        message (str): the private key used on blockchain transaction
        kem_algo (str): the KEM algorithm
    Return:
        See below.
    """

    # Initialize KEM (sender)
    sender = oqs.KeyEncapsulation(kem_algo)

    # Sender generates the key pair
    public_key = sender.generate_keypair()
    secret_key = sender.export_secret_key()

    # Sender generates ciphertext and key
    ciphertext, key = sender.encap_secret(public_key)

    # Transform the key to a symmetric key (p.s. info may lose here)
    salt = os.urandom(16)
    key_ = key_transform(key, salt)

    # Encrypt the private key used on blockchain by the transformed key
    fernet = Fernet(key_)
    message_encrypted = fernet.encrypt(message.encode())

    # Notice that the key is never saved
    return ciphertext, public_key, secret_key, salt, message_encrypted


def hybrid_decryption(kem_algo: str='Kyber512',
                      secret_key: bytes=b'\x13"l',
                      ciphertext: bytes=b'\xbb',
                      salt: bytes=b'u\xff',
                      message_encrypted: bytes=b'8964'):
    """
    This function decrypts the message encrypted using KEM
    (Key Encryption Mechanism) and SKE (Symmetric Key Encryption).

    Inputs:
        See the above.
    Return:
        The private key (string with length of 64).
    """

    receiver = oqs.KeyEncapsulation(kem_algo, secret_key)
    fernet = Fernet(key_transform(receiver.decap_secret(ciphertext), salt))
    return fernet.decrypt(message_encrypted).decode()


In [ ]:
def safebox(user_id: str='leviathan',
            message: str='8964',
            kem_algo: str='Kyber512',):
    
    # Get timing
    start_time = time.time()
    
    # Run hybrid encryption
    mistery = hybrid_encryption(message, kem_algo)
    
    # Check the directory for user
    user_dir = Path(root) / f'{user_id}'

    if not os.path.exists(user_dir):
        os.makedirs(user_dir)

    # Assign the key id for the user
    key_id = len(os.listdir(user_dir))

    # The user dict saved in our database
    user_dict = {'user_id': user_id,
                 'key_id': key_id,
                 'kem_algo': kem_algo,
                 'ciphertext_abs': bytes_to_str(mistery[0])[:32],
                 'ciphertext': bytes_to_str(mistery[0]),
                 'public_key': bytes_to_str(mistery[1]),
                 'secret_key': bytes_to_str(mistery[2]),
                 'salt': bytes_to_str(mistery[3]),
                 'message_encrypted': bytes_to_str(mistery[4])}
    
    # Order the dict
    user_dict = OrderedDict(user_dict)
    user_dict_local = OrderedDict(user_dict_local)

    # Create the file objects
    file = open(user_dir / f'{key_id}.json', 'w')
    file_local = open(user_dir / f'{key_id}_local.json', 'w')

    # Dump the data into the file
    json.dump(user_dict, file, indent=4)
    json.dump(user_dict_local, file_local, indent=4)
    file.close()
    file_local.close()

    # Leave user a message
    pprint(user_dict_local)
    print(f'Time elapsed: {time.time() - start_time:.3f}s')

In [ ]:
    start_time = time.time()

    # Run hybrid encryption
    mistery = utils.hybrid_encryption(message, kem_algo)

    # Check the directory for user
    user_dir = Path(root) / f'{user_id}'

    if not os.path.exists(user_dir):
        os.makedirs(user_dir)

    # Assign the key id for the user
    key_id = len(os.listdir(user_dir))

    # The user dict saved in our database
    user_dict = {'user_id': user_id,
                 'key_id': key_id,
                 'kem_algo': kem_algo,
                 'ciphertext_abs': utils.bytes_to_str(mistery[0])[:32],
                 'ciphertext': utils.bytes_to_str(mistery[0]),
                 'public_key': utils.bytes_to_str(mistery[1]),
                 'secret_key': utils.bytes_to_str(mistery[2]),
                 'salt': utils.bytes_to_str(mistery[3]),
                 'message_encrypted': utils.bytes_to_str(mistery[4])}

    # The user dict saved by the user
    user_dict_local = {i: user_dict[i] for i in
                       ['user_id', 'key_id', 'kem_algo', 'ciphertext_abs']}

    # Order the dict
    user_dict = OrderedDict(user_dict)
    user_dict_local = OrderedDict(user_dict_local)

    # Create the file objects
    file = open(user_dir / f'{key_id}.json', 'w')
    file_local = open(user_dir / f'{key_id}_local.json', 'w')

    # Dump the data into the file
    json.dump(user_dict, file, indent=4)
    json.dump(user_dict_local, file_local, indent=4)
    file.close()
    file_local.close()

    # Leave user a message
    pprint(user_dict_local)
    print(f'Time elapsed: {time.time() - start_time:.3f}s')